This is a starter notebook for the [Kitchenware classification](https://www.kaggle.com/competitions/kitchenware-classification) competition on Kaggle

To get started:

- Join the competition and accept rules
- Download your Kaggle credentials file
- If you're running in Saturn Cloud, configure your instance to have access to access the kaggle credentials

When this is done, we can download the data. We need to execute the following cell only once

In [1]:
!kaggle competitions download -c kitchenware-classification
!mkdir data
!unzip kitchenware-classification.zip -d data > /dev/null
!rm kitchenware-classification.zip

'kaggle' is not recognized as an internal or external command,
operable program or batch file.
A subdirectory or file data already exists.
The system cannot find the path specified.
'rm' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
!ls

'ls' is not recognized as an internal or external command,
operable program or batch file.


Now let's train a baseline model

In [3]:
import mlflow

import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow import keras

In [4]:
DATASET_SIZE = 9367
IMAGE_SIZE = 224
BATCH_SIZE = 8
WORKERS = 4
EPOCHS = 10

BASE_PATH='./data'

classes = [
    'cup', 
    'fork', 
    'glass', 
    'knife', 
    'plate', 
    'spoon'
]

First, we will load the training dataframe and split it into train and validation

In [5]:
df_train_full = pd.read_csv('data/train.csv', dtype={'Id': str})
df_train_full['filename'] = 'data/images/' + df_train_full['Id'] + '.jpg'
df_train_full.head()

,Id,label,filename
0,0560,glass,data/images/0560.jpg
1,4675,cup,data/images/4675.jpg
2,0875,glass,data/images/0875.jpg
3,4436,spoon,data/images/4436.jpg
4,8265,plate,data/images/8265.jpg


In [6]:
val_cutoff = int(len(df_train_full) * 0.8)
df_train = df_train_full[:val_cutoff]
df_val = df_train_full[val_cutoff:]

Now let's create image generators

In [7]:
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.xception import preprocess_input

from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [8]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_dataframe(
    df_train,
    x_col='filename',
    y_col='label',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

val_generator = val_datagen.flow_from_dataframe(
    df_val,
    x_col='filename',
    y_col='label',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
)

Found 4447 validated image filenames belonging to 6 classes.
Found 1112 validated image filenames belonging to 6 classes.


In [9]:
classes = np.array(list(train_generator.class_indices.keys()))
classes

array(['cup', 'fork', 'glass', 'knife', 'plate', 'spoon'], dtype='<U5')

In [10]:
earlystopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy',
                                                 min_delta = 1e-4,
                                                 patience = 5,
                                                 mode = 'max',
                                                 restore_best_weights = True,
                                                 verbose = 1)

callbacks = [earlystopping]

In [11]:
EXP_NAME = 'model_search_transformers_large'
mlflow.set_experiment(EXP_NAME)
mlflow.set_tracking_uri("sqlite:///mlruns.db")
mlflow.tensorflow.autolog()

2022/12/15 10:45:04 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of tensorflow. If you encounter errors during autologging, try upgrading / downgrading tensorflow to a supported version, or try upgrading MLflow.


In [12]:
from keras_cv_attention_models import beit, efficientnet, coatnet, davit, convnext, hornet, swin_transformer_v2, maxvit

In [13]:
base_model = efficientnet.EfficientNetV2L(  # EfficientNetV1L2(
    pretrained='imagenet21k-ft1k',  #weights='imagenet',
    input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3),
    num_classes=len(classes)
)
base_model.trainable = False
base_model = tf.keras.Model(inputs=base_model.layers[1].input, outputs= base_model.layers[-4].output)

inputs = keras.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
model = tf.keras.Sequential([
    inputs,
    base_model,
    tf.keras.layers.Flatten(),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(.2),
    tf.keras.layers.Dense(12, 'gelu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(.2),
    tf.keras.layers.Dense(6, 'softmax')
],
name = 'EfficientNetV2XL')

c:\Users\andre\anaconda3\envs\keras-attention-models\lib\site-packages\keras\initializers\initializers_v2.py:120: UserWarning: The initializer VarianceScaling is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


>>>> Load pretrained from: C:\Users\andre\.keras\models/efficientnetv2\efficientnetv2-l-21k-ft1k.h5


In [14]:
learning_rate = 0.01
optimizer = keras.optimizers.Adam(learning_rate=learning_rate)

loss = keras.losses.CategoricalCrossentropy()

model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [15]:
history = model.fit(
    x = train_generator,
    validation_data=val_generator,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    workers=WORKERS,
    callbacks=callbacks
)

Epoch 1/10
556/556 [==============================] - 66s 86ms/step - loss: 0.3551 - accuracy: 0.8871 - val_loss: 0.1536 - val_accuracy: 0.9541
Epoch 2/10
556/556 [==============================] - 42s 76ms/step - loss: 0.1193 - accuracy: 0.9660 - val_loss: 0.1358 - val_accuracy: 0.9658
Epoch 3/10
556/556 [==============================] - 42s 75ms/step - loss: 0.0926 - accuracy: 0.9685 - val_loss: 0.1582 - val_accuracy: 0.9613
Epoch 4/10
556/556 [==============================] - 42s 76ms/step - loss: 0.0892 - accuracy: 0.9739 - val_loss: 0.1504 - val_accuracy: 0.9703
Epoch 5/10
556/556 [==============================] - 42s 75ms/step - loss: 0.0820 - accuracy: 0.9728 - val_loss: 0.1480 - val_accuracy: 0.9676
Epoch 6/10
556/556 [==============================] - 42s 75ms/step - loss: 0.0554 - accuracy: 0.9825 - val_loss: 0.1642 - val_accuracy: 0.9703
Epoch 7/10
556/556 [==============================] - 42s 75ms/step - loss: 0.0837 - accuracy: 0.9739 - val_loss: 0.1403 - val_accuracy:

2022/12/15 10:51:56 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during tensorflow autologging: No Experiment with id=577318239482065117 exists
